# Build a BERT SageMaker Pipeline

https://github.com/kubeflow/pipelines/blob/master/samples/contrib/aws-samples/mnist-kmeans-sagemaker/mnist-classification-pipeline.py

https://github.com/aws-samples/eks-kubeflow-workshop/blob/master/notebooks/05_Kubeflow_Pipeline/05_04_Pipeline_SageMaker.ipynb

## Install AWS Python SDK (`boto3`)

In [ ]:
!pip install boto3

## Install Kubeflow Pipelines SDK

In [ ]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade

In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3

AWS_REGION_AS_SLIST=!curl -s http://169.254.169.254/latest/meta-data/placement/availability-zone | sed 's/\(.*\)[a-z]/\1/'
AWS_REGION = AWS_REGION_AS_SLIST.s
print('Region: {}'.format(AWS_REGION))

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')
print('Account ID: {}'.format(AWS_ACCOUNT_ID))

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

## Copy `data` and `valid_data.csv` into your S3 bucket.

In [ ]:
!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/data s3://$S3_BUCKET/mnist_kmeans_example/data
!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/input/valid_data.csv s3://$S3_BUCKET/mnist_kmeans_example/input/

# Build Pipeline

In [ ]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

In [ ]:
sagemaker_hpo_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/3ebd075212e0a761b982880707ec497c36a99d80/components/aws/sagemaker/hyperparameter_tuning/component.yaml')


In [ ]:
sagemaker_process_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/3ebd075212e0a761b982880707ec497c36a99d80/components/aws/sagemaker/process/component.yaml')


In [ ]:
sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/3ebd075212e0a761b982880707ec497c36a99d80/components/aws/sagemaker/train/component.yaml')


In [ ]:
sagemaker_model_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/3ebd075212e0a761b982880707ec497c36a99d80/components/aws/sagemaker/model/component.yaml')


In [ ]:
sagemaker_deploy_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/3ebd075212e0a761b982880707ec497c36a99d80/components/aws/sagemaker/deploy/component.yaml')


# Create Pipeline

We will create a training job first. Once training job is done, it will persist trained model to S3. 

Then a job will be kicked off to create a `Model` manifest in Sagemaker. 

With this model, batch transformation job can use it to predict on other datasets, prediction service can create an endpoint using it.


> Note: remember to use your **role_arn** to successfully run the job.

> Note: If you use a different region, please replace `us-west-2` with your region. 

> Note: ECR Images for k-means algorithm

|Region| ECR Image|
|------|----------|
|us-west-1|632365934929.dkr.ecr.us-west-1.amazonaws.com|
|us-west-2|174872318107.dkr.ecr.us-west-2.amazonaws.com|
|us-east-1|382416733822.dkr.ecr.us-east-1.amazonaws.com|
|us-east-2|404615174143.dkr.ecr.us-east-2.amazonaws.com|
|us-gov-west-1|226302683700.dkr.ecr.us-gov-west-1.amazonaws.com|
|ap-east-1|286214385809.dkr.ecr.ap-east-1.amazonaws.com|
|ap-northeast-1|351501993468.dkr.ecr.ap-northeast-1.amazonaws.com|
|ap-northeast-2|835164637446.dkr.ecr.ap-northeast-2.amazonaws.com|
|ap-south-1|991648021394.dkr.ecr.ap-south-1.amazonaws.com|
|ap-southeast-1|475088953585.dkr.ecr.ap-southeast-1.amazonaws.com|
|ap-southeast-2|712309505854.dkr.ecr.ap-southeast-2.amazonaws.com|
|ca-central-1|469771592824.dkr.ecr.ca-central-1.amazonaws.com|
|eu-central-1|664544806723.dkr.ecr.eu-central-1.amazonaws.com|
|eu-north-1|669576153137.dkr.ecr.eu-north-1.amazonaws.com|
|eu-west-1|438346466558.dkr.ecr.eu-west-1.amazonaws.com|
|eu-west-2|644912444149.dkr.ecr.eu-west-2.amazonaws.com|
|eu-west-3|749696950732.dkr.ecr.eu-west-3.amazonaws.com|
|me-south-1|249704162688.dkr.ecr.me-south-1.amazonaws.com|
|sa-east-1|855470959533.dkr.ecr.sa-east-1.amazonaws.com|

In [ ]:
SAGEMAKER_ROLE_ARN='arn:aws:iam::{}:role/TeamRole'.format(AWS_ACCOUNT_ID)

# Configure your s3 bucket.
S3_PIPELINE_PATH='s3://{}/bert-kubeflow-pipeline'.format(S3_BUCKET)

# TODO:  Implement the other region checks
if AWS_REGION == 'us-west-2':
    AWS_ECR_REGISTRY='174872318107.dkr.ecr.us-west-2.amazonaws.com'

if AWS_REGION == 'us-east-1':
    AWS_ECR_REGISTRY='382416733822.dkr.ecr.us-east-1.amazonaws.com'

# Setup Pre-Processing Code 

In [ ]:
processing_code_s3_uri = 's3://{}/processing_code/kmeans_preprocessing.py'.format(S3_BUCKET)
print(processing_code_s3_uri)

!aws s3 cp ./kmeans_preprocessing.py $processing_code_s3_uri


# Setup Training Code

In [ ]:
!tar -cvzf sourcedir.tar.gz -C code .

In [ ]:
training_code_s3_uri = 's3://{}/training_code/'.format(S3_BUCKET)
print(training_code_s3_uri)

!aws s3 cp sourcedir.tar.gz $training_code_s3_uri

In [ ]:
def processing_input(input_name, s3_uri, local_path):
    return {
        "InputName": input_name,
        "S3Input": {
            "S3Uri": s3_uri,
            "LocalPath": local_path,
            "S3DataType": "S3Prefix",
            "S3InputMode": "File",
        },
    }

def processing_output(output_name, s3_uri, local_path):
    return {
        "OutputName": output_name,
        "S3Output": {
            "S3Uri": s3_uri,
            "LocalPath": local_path,
            "S3UploadMode": "EndOfJob",
        },
    }

In [ ]:
def training_input(input_name, s3_uri):
    return {
        "ChannelName": input_name,
        "DataSource": {"S3DataSource": {"S3Uri": s3_uri, "S3DataType": "S3Prefix"}},
    }

In [ ]:
@dsl.pipeline(
    name="MNIST Classification pipeline",
    description="MNIST Classification using KMEANS in SageMaker",
)
def mnist_classification(role_arn=SAGEMAKER_ROLE_ARN, bucket_name=S3_BUCKET):
    region = "us-east-1"
    processing_instance_type = 'ml.c5.2xlarge'
    processing_instance_count = 2    
    training_instance_type = 'ml.c5.4xlarge'
    training_instance_count = 1
    deploy_instance_type = 'ml.m5.4xlarge'
    deploy_instance_count = 1
    processing_image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04'
    training_image = "382416733822.dkr.ecr.us-east-1.amazonaws.com/kmeans:1"
#    training_image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04'    
#    training_image = '763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:1.15.2-gpu-py36-cu100-ubuntu18.04'

#    serving_image='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:1.15.2-cpu'

    # Training input and output location based on bucket name
    process = sagemaker_process_op(
        role=role_arn,
        region=region,
        image=processing_image, #"763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-cpu-py36-ubuntu16.04",
        instance_type=instance_type,
        container_entrypoint=[
            "python",
            "/opt/ml/processing/code/kmeans_preprocessing.py",
        ],
        input_config=[
            processing_input(
                "mnist_tar",
                "s3://sagemaker-sample-data-us-east-1/algorithms/kmeans/mnist/mnist.pkl.gz",
                "/opt/ml/processing/input",
            ),
            processing_input(
                "source_code",
                "{}".format(processing_code_s3_uri),
                "/opt/ml/processing/code",
            ),
        ],
        output_config=[
            processing_output(
                "train_data",
                f"s3://{bucket_name}/mnist_kmeans_example/",
                "/opt/ml/processing/output_train/",
            ),
            processing_output(
                "test_data",
                f"s3://{bucket_name}/mnist_kmeans_example/",
                "/opt/ml/processing/output_test/",
            ),
            processing_output(
                "valid_data",
                f"s3://{bucket_name}/mnist_kmeans_example/input/",
                "/opt/ml/processing/output_valid/",
            ),
        ],
    )
    
    train_channels = [
        training_input("train", f"s3://{bucket_name}/mnist_kmeans_example/train_data")
    ]
    train_output_location = f"s3://{bucket_name}/mnist_kmeans_example/output"

    # .after(process) is explicitly appended below
    training = sagemaker_train_op(
        region=region,
        image=training_image,
        hyperparameters={"k": "10", 
                         "feature_dim": "784", 
                         "mini_batch_size": "500", 
                         "extra_center_factor": "10", 
                         "init_method": "random",
                        },
        channels=train_channels,
        instance_type=instance_type,
        model_artifact_path=train_output_location,
        role=role_arn,
    ).after(process)

    # .after(process) is implied because we depend on training.outputs[]
    create_model = sagemaker_model_op(
        region=region,
        model_name=training.outputs["job_name"],
        image=serving_image, # training.outputs["training_image"],
        model_artifact_url=training.outputs["model_artifact_url"],
        role=role_arn,
    )

    # .after(process) is implied because we depend on create_model.outputs
    sagemaker_deploy_op(
        region=region,
        model_name_1=create_model.output,
        instance_type=deploy_instance_type
    )


# 4. Compile your pipeline

In [ ]:
kfp.compiler.Compiler().compile(mnist_classification, 'mnist-classification-pipeline.zip')

In [ ]:
!ls -al ./mnist-classification-pipeline.zip

In [ ]:
!unzip -o ./mnist-classification-pipeline.zip

In [ ]:
!cat pipeline.yaml

# 5. Deploy your pipeline

In [ ]:
client = kfp.Client()

aws_experiment = client.create_experiment(name='aws')

my_run = client.run_pipeline(aws_experiment.id, 
                             'mnist-classification-pipeline', 
                             'mnist-classification-pipeline.zip')

## Training

_Note:  The above training job may take 5-10 minutes.  Please be patient._

In the meantime, open the SageMaker Console to monitor the progress of your training job.

![SageMaker Training Job Console](img/sagemaker-training-job-console.png)

## Get the Name of the Deployed Prediction Endpoint
First, we need to get the endpoint name of our newly-deployed SageMaker Prediction Endpoint.

Open AWS console and enter SageMaker service, find the endpoint name as the following picture shows.

![download-pipeline](images/sm-endpoint.jpg)

# Make a Prediction

# _YOU MUST COPY/PASTE THE `ENDPOINT_NAME` BEFORE CONTINUING_
Make sure to include preserve the single-quotes as shown below.

In [ ]:
import pickle, gzip, numpy, urllib.request, json
from urllib.parse import urlparse
import json
import io
import boto3

#################################
#################################
# Replace ENDPOINT_NAME with the endpoint name in the SageMaker console.
# Surround with single quotes.
ENDPOINT_NAME= # 'Endpoint-<your-endpoint-name>'
#################################
#################################

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

# Simple function to create a csv from our numpy array
def np2csv(arr):
    csv = io.BytesIO()
    numpy.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

runtime = boto3.Session(region_name=AWS_REGION).client('sagemaker-runtime')

payload = np2csv(train_set[0][30:31])

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
print(result)

## Clean up

Go to Sagemaker console and delete `endpoint` and `model`.